In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
RE = "Solar_PBE" # Solar_PBE Wind_Wallonie_Elia
address = "https://raw.githubusercontent.com/Jaeik-Jeong/DeepBid/main/"

data_train_csv1 = pd.read_csv(address+RE+'_16.csv', index_col=0)
data_train_csv2 = pd.read_csv(address+RE+'_17.csv', index_col=0)
data_train_csv  = pd.concat([data_train_csv1, data_train_csv2])
data_val_csv    = pd.read_csv(address+RE+'_18.csv', index_col=0)
data_test_csv   = pd.read_csv(address+RE+'_19.csv', index_col=0)

data_price = pd.read_csv(address+'Price_Elia_Imbalance_16_19.csv', index_col=0)
data_train_csv['Price(€)'] = data_price['Positive imbalance price'][:len(data_train_csv)]
data_val_csv['Price(€)']   = data_price['Positive imbalance price'][len(data_train_csv):len(data_train_csv)+len(data_val_csv)]
data_test_csv['Price(€)']  = data_price['Positive imbalance price'][len(data_train_csv)+len(data_val_csv):]

In [ ]:
# Data Preprocessing
 
Battery_Size = 0.15 #p.u.
unit         = 1 #unit: 15 minute
 
RE_Capacity1 = max(data_train_csv['Power(MW)'])
RE_Capacity2 = max(data_val_csv['Power(MW)'])
RE_Capacity3 = max(data_test_csv['Power(MW)'])
max_price = max(data_price['Marginal incremental price'])
 
size_train0 = int(len(data_train_csv)/unit)
size_val0   = int(len(data_val_csv)/unit)
size_test0  = int(len(data_test_csv)/unit)
 
data_train0 = []; data_train = []; price_train0 = []; price_train = [];
for i in range(size_train0):
    data_train0  += [round(pd.Series.mean(data_train_csv['Power(MW)'][i*unit:(i+1)*unit])/RE_Capacity1, 3)]
    price_train0 += [round(pd.Series.mean(data_train_csv['Price(€)'][i*unit:(i+1)*unit])/max_price, 3)]
    if data_train0[i] > 0: data_train += [data_train0[i]]; price_train += [price_train0[i]]
 
data_val0 = []; data_val = []; price_val0 = []; price_val = []
for i in range(size_val0):
    data_val0  += [round(pd.Series.mean(data_val_csv['Power(MW)'][i*unit:(i+1)*unit])/RE_Capacity2, 3)]
    price_val0 += [round(pd.Series.mean(data_val_csv['Price(€)'][i*unit:(i+1)*unit])/max_price, 3)]
    if data_val0[i] > 0: data_val += [data_val0[i]]; price_val += [price_val0[i]]
 
data_test0 = []; data_test = []; price_test0 = []; price_test = []
for i in range(size_test0):
    data_test0  += [round(pd.Series.mean(data_test_csv['Power(MW)'][i*unit:(i+1)*unit])/RE_Capacity3, 3)]
    price_test0 += [round(pd.Series.mean(data_test_csv['Price(€)'][i*unit:(i+1)*unit])/max_price, 3)]
    if data_test0[i] > 0: data_test += [data_test0[i]]; price_test += [price_test0[i]]

In [ ]:
# LSTM

n_layers       = 2
in_size        = 1
hidden_size    = 64
out_size       = 1
batch_size     = 128
learning_rate  = 0.001

class LSTM(nn.Module):
    def __init__(self):
        super(LSTM, self).__init__()
        self.fc_in  = nn.Linear(in_size, hidden_size)
        self.rnn    = nn.LSTM(hidden_size, hidden_size, n_layers, batch_first=True)
        self.fc_out = nn.Linear(hidden_size, out_size)
    
    def forward(self, x, hidden):
        x = F.relu(self.fc_in(x))
        x = x.view(1, -1, hidden_size)
        x, hidden = self.rnn(x, hidden)
        out = self.fc_out(x)
        out = F.relu(out.view(-1, out_size))
        return out, hidden
        
def train_net(model, batch, optimizer):
    x, h, y = batch[0], batch[1], batch[2]
    loss = F.mse_loss(model.forward(x, h)[0], y)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

In [ ]:
# Training LSTM

total_epoch    = 100
print_interval = 1

model = LSTM()
size_train = len(data_train)
size_val = len(data_val)
size_test = len(data_test)

train_input = np.zeros((size_train-1, 1))
train_output = np.zeros((size_train-1, 1))
for i in range(size_train-1):
    train_input[i,:] = data_train[i]
    train_output[i,:] = data_train[i+1]

val_input = np.zeros((size_val-1, 1))
val_output = np.zeros((size_val-1, 1))
for i in range(size_val-1):
    val_input[i,:] = data_val[i]
    val_output[i,:] = data_val[i+1]

test_input = np.zeros((size_test-1, 1))
test_output = np.zeros((size_test-1, 1))
for i in range(size_test-1):
    test_input[i,:] = data_test[i]
    test_output[i,:] = data_test[i+1]

total_batch = int((size_train-1)/batch_size) + 1
pred_train, pred_val, pred_test = [], [], [] # Predicted Value
mae_train,  mae_val,  mae_test  = [], [], [] # Mean Absolute Error

hidden = (torch.zeros([n_layers, 1, hidden_size], dtype=torch.float), torch.zeros([n_layers, 1, hidden_size], dtype=torch.float))
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
for epoch in range(total_epoch):
    for i in range(total_batch):
        batch_x = torch.tensor(train_input[batch_size*i:batch_size*(i+1),:] ,dtype=torch.float)
        batch_y = torch.tensor(train_output[batch_size*i:batch_size*(i+1),:] ,dtype=torch.float)
        batch = [batch_x, hidden, batch_y]
        train_net(model, batch, optimizer)
        _, hidden = model.forward(batch_x, hidden)
        hidden = (hidden[0].detach(), hidden[1].detach())

    hidden = (torch.zeros([n_layers, 1, hidden_size], dtype=torch.float), torch.zeros([n_layers, 1, hidden_size], dtype=torch.float))
    if epoch == 0 or (epoch+1) % print_interval == 0:
        train_predict = model.forward(torch.tensor(train_input, dtype=torch.float), hidden)[0].detach().numpy()
        pred_train += [list(train_predict.flatten())]
        mae_train  += [list(np.abs(train_predict - train_output).flatten())]
        
        val_predict = model.forward(torch.tensor(val_input, dtype=torch.float), hidden)[0].detach().numpy()
        pred_val += [list(val_predict.flatten())]
        mae_val  += [list(np.abs(val_predict - val_output).flatten())]
        
        test_predict = model.forward(torch.tensor(test_input, dtype=torch.float), hidden)[0].detach().numpy()
        pred_test += [list(test_predict.flatten())]
        mae_test  += [list(np.abs(test_predict - test_output).flatten())]

        MAE_train = round(100*np.mean(mae_train[-1]),2)
        MAE_val   = round(100*np.mean(mae_val[-1]),2)
        MAE_test  = round(100*np.mean(mae_test[-1]),2)

        print("epoch: {}".format(epoch+1))
        print("MAE_train: {}%".format(MAE_train).ljust(25), end="")
        print("MAE_val: {}%".format(MAE_val).ljust(25), end="")
        print("MAE_test: {}%".format(MAE_test).ljust(25))
        print("------------------------------------------------------------------------------------------")

epoch: 1
MAE_train: 2.41%         MAE_val: 2.65%           MAE_test: 2.45%          
------------------------------------------------------------------------------------------
epoch: 2
MAE_train: 2.01%         MAE_val: 2.16%           MAE_test: 2.04%          
------------------------------------------------------------------------------------------
epoch: 3
MAE_train: 2.0%          MAE_val: 2.21%           MAE_test: 2.04%          
------------------------------------------------------------------------------------------
epoch: 4
MAE_train: 2.04%         MAE_val: 2.27%           MAE_test: 2.08%          
------------------------------------------------------------------------------------------
epoch: 5
MAE_train: 2.11%         MAE_val: 2.35%           MAE_test: 2.15%          
------------------------------------------------------------------------------------------


KeyboardInterrupt: ignored

In [ ]:
# Environment

E_max   = Battery_Size
P_max   = E_max
tdelta  = unit/4
soc_min = 0.1
soc_max = 0.9
a0 = -1.031; a1 = 35; a2 = 3.685; a3 = 0.2156; a4 = 0.1178; a5 = 0.3201
b0 = 0.1463; b1 = 30.27; b2 = 0.1037; b3 = 0.0584; b4 = 0.1747; b5 = 0.1288
c0 = 0.1063; c1 = 62.49; c2 = 0.0437; d0 = 0.0712; d1 = 61.4; d2 = 0.0288
N = 130*215*E_max/0.1
beta = 10/max_price
 
select_num = np.argmin(np.mean(mae_val,axis=1))
select_train = np.array(pred_train[select_num][:])
select_val = np.array(pred_val[select_num][:])
select_test = np.array(pred_test[select_num][:])
select_test_real = np.array(data_test[1:])
select_test_price = np.array(price_test[1:])
 
E = E_max/2
mbe = []
reward = []
info = []
for i in range(len(select_test)):
    bid = select_test[i]
    gen = select_test_real[i]
    rat = 1
    imb = select_test_price[i]
    
    soc = E/E_max
    Voc = a0*np.exp(-a1*soc) + a2 + a3*soc - a4*soc**2 + a5*soc**3
    Rs  = b0*np.exp(-b1*soc) + b2 + b3*soc - b4*soc**2 + b5*soc**3
    Rts = c0*np.exp(-c1*soc) + c2
    Rtl = d0*np.exp(-d1*soc) + d2
    R   = Rs + Rts + Rtl
 
    I_cmax = 1000000*E_max*(soc_max - soc)/N/(Voc*tdelta)
    I_dmax = 1000000*E_max*(soc - soc_min)/N/(Voc*tdelta)
    p_cmax = N*(Voc*I_cmax + I_cmax**2*R)
    p_dmax = N*(Voc*I_dmax - I_dmax**2*R)
 
    P_cmax = p_cmax/1000000; P_dmax = p_dmax/1000000
    P_c = min(max(rat*(gen-bid), 0), P_max, P_cmax)
    P_d = min(max(rat*(bid-gen), 0), P_max, P_dmax)
    p_c = 1000000*P_c/N; p_d = 1000000*P_d/N
 
    I_c = -(Voc - np.sqrt(Voc**2 + 4*R*p_c))/(2*R)
    I_d = (Voc - np.sqrt(Voc**2 - 4*R*p_d))/(2*R)
    if not np.isclose(p_c, 0):
        eff_c = (Voc*I_c)/p_c
        E = E + eff_c*P_c*tdelta
        disp = gen - P_c
        info += [[gen, round(bid,4), 'C', round(P_c,4), round(disp,4), round(eff_c,4), round(E,4)]]
    elif not np.isclose(p_d, 0):
        eff_d = p_d/(Voc*I_d)
        E = E - (1/eff_d)*P_d*tdelta
        disp = gen + P_d
        info += [[gen, round(bid,4), 'D', round(P_d,4), round(disp,4), round(eff_d,4), round(E,4)]]
    else:
        disp = gen
        info += [[gen, round(bid,4), 'N', 'N', round(disp,4), 'N', round(E,4)]]
    
    mbe += [abs(bid - disp)]
    reward += [(imb*disp - imb*abs(bid-disp) - beta*(P_c+P_d))*tdelta]
 
MAE_test = round(100*np.mean(np.abs(select_test_real - select_test)),2)
MBE_test = round(100*np.mean(mbe),2)
print("MAE_test: {}%".format(MAE_test))
print("MBE_test: {}%".format(MBE_test))
print("REV_test: ${}".format(round(max_price*RE_Capacity3*np.mean(reward),3)))

# pd.DataFrame(select_train).to_csv(RE+"_Model1_train.csv")
# pd.DataFrame(select_val).to_csv(RE+"_Model1_val.csv")
# pd.DataFrame(select_test).to_csv(RE+"_Model1_FB.csv")

MAE_test: 2.04%
MBE_test: 1.66%
REV_test: $141.548
